In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import ticker as mticker
import sys
import warnings
warnings.filterwarnings('ignore')
#np.set_printoptions(precision = 10, threshold = sys.maxsize)
#np.set_printoptions(linewidth = np.inf)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.expand_frame_repr', False)
#pd.set_option('max_colwidth', None)
pd.options.display.float_format = '{:,.10f}'.format

In [3]:
import multiprocessing
import psutil

psutil.cpu_count(logical=False)

num_processors = multiprocessing.cpu_count()
workers = num_processors-1
print(f'Available CPUs: {num_processors}')

Available CPUs: 12


In [4]:
import os
from pandarallel import pandarallel

num_processors = os.cpu_count()
if num_processors is not None:
    num_processors = max(1, num_processors - 1)

pandarallel.initialize(nb_workers=6, use_memory_fs=False)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [6]:
%%time

df_news_final_project = pd.read_parquet('https://storage.googleapis.com/msca-bdp-data-open/news_final_project/news_final_project.parquet', engine='pyarrow')
df_news_final_project.shape

In [6]:
from lingua import Language, LanguageDetectorBuilder

detector = LanguageDetectorBuilder.from_all_spoken_languages().build()
confidence_values = detector.compute_language_confidence_values_in_parallel(df_news_final_project['title'])

eng_lang = []
for confidence in confidence_values:
    if confidence[0].value>0.5:
        eng_lang.append(1)
    else:
        eng_lang.append(0)

df_news_final_project['eng_lang'] = eng_lang
df_news_final_project.eng_lang.value_counts(normalize=True)
df_news_final_project = df_news_final_project[df_news_final_project['eng_lang']==1]
df_news_final_project.shape

(162744, 6)

In [ ]:
import re
import string
from nltk.corpus import stopwords

def clean_text(text):
    cleaned_text = re.sub(r"\b[A-Z][a-z]+[A-Z][a-z]+\b", "", text)
    #cleaned_text = re.sub(r'"[^"]*"', '', text)
    #cleaned_text = re.sub(r'http\S+', '', text)
    #cleaned_text = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", '', text)
    #cleaned_text = re.sub(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', "", text)
    #cleaned_text = re.sub(r'\d+', '', text)
    #cleaned_text = re.sub(r'\B[%@#+→?©<>]\w+', '', text)
    cleaned_text = re.sub(r"\s+", " ", cleaned_text).strip()
    stop_words = set(stopwords.words('english'))
    cleaned_text = ' '.join([word for word in text.split() if word.lower() not in stop_words])
    return cleaned_text

df_news_final_project['clean_text'] = df_news_final_project['text'].parallel_apply(clean_text)

In [16]:
import pandas as pd

df_news_final_project['word_count'] = df_news_final_project['text'].apply(lambda x: len(x.split()))
df = df_news_final_project[df_news_final_project['word_count']>50]

In [14]:
df = df_news_final_project[df_news_final_project['word_count']>50]

In [18]:
import re
import string
from nltk.corpus import stopwords

def clean_text(text):
    cleaned_text = re.sub(r"\b[A-Z][a-z]+[A-Z][a-z]+\b", "", text)
    #cleaned_text = re.sub(r'"[^"]*"', '', text)
    #cleaned_text = re.sub(r'http\S+', '', text)
    #cleaned_text = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", '', text)
    #cleaned_text = re.sub(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', "", text)
    #cleaned_text = re.sub(r'\d+', '', text)
    #cleaned_text = re.sub(r'\B[%@#+→?©<>]\w+', '', text)
    cleaned_text = re.sub(r"\s+", " ", cleaned_text).strip()
    stop_words = set(stopwords.words('english'))
    cleaned_text = ' '.join([word for word in text.split() if word.lower() not in stop_words])
    return cleaned_text

df['clean_text'] = df['text'].apply(clean_text)

In [20]:
df.head()

url  \
0  http://galusaustralis.com/2020/02/486473/legaltech-artificial-intelligence-market-2019-technology-advancement-and-future-scope-casetext-inc-catalyst-repository-systems-ebrevia/   
1                                          http://newsparliament.com/2020/02/27/children-with-autism-saw-their-learning-and-social-skills-boosted-after-playing-with-this-ai-robot/   
2                                                                                                                             http://www.agoravox.it/Covid-19-un-messaggio-dai.html   
3              http://www.desototimes.com/news/child-actor-claude-jarman-jr-looks-back-on-his-movie-career-during-the-golden-days/article_0b310fae-e767-11ec-982e-a380305dda2f.html   
4                                                                  http://www.millenniumpost.in/big-stories/ai-express-aircraft-skids-off-runaway-in-kozhikode-rescue-ops-on-414926   

         date language  \
0  2020-02-26       en   
1  2020-02-27       en   
2  2020-03-13       en   
3  2022-06-09       en   
4  2020-08-07       en   

                                                                                                                                                           title  \
0  LegalTech Artificial Intelligence Market 2019 Technology Advancement and Future Scope – Casetext Inc., Catalyst Repository Systems, eBREVIA – Galus Australis   
1                                           Children With Autism Saw Their Learning and Social Skills Boosted After Playing With This AI Robot – News Parliament   
2                                                                        Covid-19: un messaggio dai ricercatori italiani ai colleghi stranieri - AgoraVox Italia   
3                                    Child actor Claude Jarman Jr. looks back on his movie career during the “Golden Days” of Hollywood | News | desototimes.com   
4                                                                                                      Two dead as AI Express flight skids off Kozhikode airport   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [34]:
from cleantext import clean

def clean_text_v2(text):
    return clean(text,
        fix_unicode=True,              
        to_ascii=True,              
        lower=False,                  
        no_line_breaks=True,         
        no_urls=True,                   
        no_emails=True,                
        no_phone_numbers=True,         
        no_numbers=True,              
        no_digits=True,                
        no_currency_symbols=True,       
        no_punct=True,                  
        lang="en")

df['clean_text_v2'] = df['clean_text'].apply(clean_text_v2)
df['clean_title'] = df['title'].apply(clean_text_v2)

In [ ]:
#web crawled data package to be checked

In [36]:
df.to_csv(r'C:\Users\Rahul\Downloads\NLP Final Project\Preprocessing_1.csv')

In [37]:
df.to_parquet(r'C:\Users\Rahul\Downloads\NLP Final Project\Preprocessing_1.parquet', engine='pyarrow')

In [38]:
df1 = pd.read_parquet(r'C:\Users\Rahul\Downloads\NLP Final Project\Preprocessing_1.parquet')

In [39]:
df1 = df1[['date', 'clean_title', 'clean_text_v2']]

In [ ]:
# <NUMBER> <CUR> <EMAIL> <PHONE>

In [40]:
df1.head()

date  \
0  2020-02-26   
1  2020-02-27   
2  2020-03-13   
3  2022-06-09   
4  2020-08-07   

                                                                                                                                                  clean_title  \
0  LegalTech Artificial Intelligence Market <NUMBER> Technology Advancement and Future Scope Casetext Inc Catalyst Repository Systems eBREVIA Galus Australis   
1                                          Children With Autism Saw Their Learning and Social Skills Boosted After Playing With This AI Robot News Parliament   
2                                                                   Covid<NUMBER> un messaggio dai ricercatori italiani ai colleghi stranieri AgoraVox Italia   
3                                     Child actor Claude Jarman Jr looks back on his movie career during the Golden Days of Hollywood | News | desototimescom   
4                                                                                                   Two dead as AI Express flight skids off Kozhikode airport   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [ ]:
#Two dead as AI Express flight skids off Kozhikode airport - to be removed

In [61]:
regex_pattern = r'<NUMBER>|<CUR>|<EMAIL>|<PHONE>|<URL>'

df1['clean_text_v3'] = df1['clean_text_v2'].str.replace(regex_pattern, '', regex=True)
df1['clean_title_v3'] = df1['clean_title'].str.replace(regex_pattern, '', regex=True)

In [48]:
import pandas as pd
from nltk.tokenize import word_tokenize
import re
import nltk

#keywords = ['machine learning', 'artificial intelligence', 'natural language processing', 
#            'gpt', 'ai', 'ml', 'nlp', 'deep learning', 'dl', 'mldl', 'language models', 'openai']

keywords = ['Data Science', 'data science', 'DATA SCIENCE','AI', 'ai', 'artificial intelligence','Artificial Intelligence',
            'ARTIFICIAL INTELLIGENCE', 'ML', 'NLP', 
            'Artificial General Intelligence','Chatbots', 'AI Marketplaces', 'Intelligent Applications', 
           'Augmented Intelligence', 'Decision Intelligence', 'AI Cloud Services', 'cloud services', 
            'GPU Accelerators', 'Computer Vision','Deep Neural Network', 'Deep Learning','Cognitive Computing',
            'Knowledge Graphs', 
            'Responsible AI', 'Machine Customers', 'Decision Intelligence',
           'Human-Centered AI', 'AI Governance', 'Natural Language Processing', 
            'Machine Learning', 'Smart Robots', 'Operational AI Systems' , 'Data-Centric AI', 'AI TRiSM', 'Generative AI',
           'Responsible AI', 'OpenAI', 'openai', 'gpt', 'GPT']

keywords_regex = '|'.join(r'\b{}\b'.format(re.escape(keyword)) for keyword in keywords)
keywords_pattern = re.compile(keywords_regex, re.IGNORECASE)

def count_keywords(text):
    tokens = word_tokenize(text)
    processed_text = ' '.join(tokens)
    matches = re.findall(keywords_pattern, processed_text)
    return len(matches)

df1['count_keywords'] = df1['clean_text_v3'].apply(count_keywords)

In [56]:
df1 = df1[df1['count_keywords']>5]

In [58]:
df1.head()

date  \
0  2020-02-26   
2  2020-03-13   
7  2021-02-25   
8  2022-10-06   
9  2020-04-29   

                                                                                                                                                  clean_title  \
0  LegalTech Artificial Intelligence Market <NUMBER> Technology Advancement and Future Scope Casetext Inc Catalyst Repository Systems eBREVIA Galus Australis   
2                                                                   Covid<NUMBER> un messaggio dai ricercatori italiani ai colleghi stranieri AgoraVox Italia   
7                                                        MulticoreWare Inc Becomes CEVAs Trusted Partner for Imaging Computer Vision Consumer Electronics Net   
8                                                                     Healthcare Artificial Intelligence Market Analysis and Forecast for Next <NUMBER> Years   
9                                        Artificial Intelligence As A Service Market Opportunities Market Analysis Outlook To <NUMBER> 0w Market News Reports   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [ ]:
#keywords = ['Cookie' 'Policy', 'LogIn', 'LogOut', 'Copyrights']

from transformers import pipeline

summarizer = pipeline("summarization")

def summarize_text(text):
    summary_text = summarizer(text, max_length=130, min_length=30, do_sample=False)
    return summary_text[0]['summary_text']

df1['text_sum'] = df1['clean_text_v3'].apply(summarize_text)

from gensim.summarization import summarize

def summarize_text(text):
    try:
        summary = summarize(text, ratio=1)
        return summary
    except ValueError:
        return text
    
df1['text_sum'] = df1['clean_text_v3'].apply(summarize_text)

In [62]:
df1.head()

date  \
0  2020-02-26   
2  2020-03-13   
7  2021-02-25   
8  2022-10-06   
9  2020-04-29   

                                                                                                                                                  clean_title  \
0  LegalTech Artificial Intelligence Market <NUMBER> Technology Advancement and Future Scope Casetext Inc Catalyst Repository Systems eBREVIA Galus Australis   
2                                                                   Covid<NUMBER> un messaggio dai ricercatori italiani ai colleghi stranieri AgoraVox Italia   
7                                                        MulticoreWare Inc Becomes CEVAs Trusted Partner for Imaging Computer Vision Consumer Electronics Net   
8                                                                     Healthcare Artificial Intelligence Market Analysis and Forecast for Next <NUMBER> Years   
9                                        Artificial Intelligence As A Service Market Opportunities Market Analysis Outlook To <NUMBER> 0w Market News Reports   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [64]:
df1 = df1[['date', 'clean_title_v3', 'clean_text_v3']]

In [65]:
df1.to_parquet(r'C:\Users\Rahul\Downloads\NLP Final Project\Preprocessing_2.parquet', engine='pyarrow')

In [5]:
df2 = pd.read_parquet(r'C:\Users\Rahul\Downloads\NLP Final Project\Preprocessing_2.parquet')

In [6]:
df2.head()

,date,clean_title_v3,clean_text_v3
0,2020-02-26,LegalTech Artificial Intelligence Market Tech...,LegalTech Artificial Intelligence Market Tech...
2,2020-03-13,Covid un messaggio dai ricercatori italiani ai...,Covid un messaggio dai ricercatori italiani ai...
7,2021-02-25,MulticoreWare Inc Becomes CEVAs Trusted Partne...,MulticoreWare Inc Becomes CEVAs Trusted Partne...
8,2022-10-06,Healthcare Artificial Intelligence Market Anal...,Healthcare Artificial Intelligence Market Anal...
9,2020-04-29,Artificial Intelligence As A Service Market Op...,Artificial Intelligence Service Market Opportu...


In [7]:
def clean_text_less14(text):
#    text = text.lower()
    words = text.split()
    cleaned_words = [word for word in words if len(word) <= 14]
    cleaned_text = ' '.join(cleaned_words)
    return cleaned_text

df2['clean_text_v4'] = df2['clean_text_v3'].apply(clean_text_less14)

In [8]:
def remove_alphanumeric_words(s):
    return ' '.join(word for word in s.split() if not any(char.isdigit() for char in word))

df2['clean_text_v5'] = df2['clean_text_v4'].apply(remove_alphanumeric_words)

In [9]:
df2.head()

,date,clean_title_v3,clean_text_v3,clean_text_v4,clean_text_v5
0,2020-02-26,LegalTech Artificial Intelligence Market Tech...,LegalTech Artificial Intelligence Market Tech...,LegalTech Artificial Intelligence Market Techn...,LegalTech Artificial Intelligence Market Techn...
2,2020-03-13,Covid un messaggio dai ricercatori italiani ai...,Covid un messaggio dai ricercatori italiani ai...,Covid un messaggio dai ricercatori italiani ai...,Covid un messaggio dai ricercatori italiani ai...
7,2021-02-25,MulticoreWare Inc Becomes CEVAs Trusted Partne...,MulticoreWare Inc Becomes CEVAs Trusted Partne...,MulticoreWare Inc Becomes CEVAs Trusted Partne...,MulticoreWare Inc Becomes CEVAs Trusted Partne...
8,2022-10-06,Healthcare Artificial Intelligence Market Anal...,Healthcare Artificial Intelligence Market Anal...,Healthcare Artificial Intelligence Market Anal...,Healthcare Artificial Intelligence Market Anal...
9,2020-04-29,Artificial Intelligence As A Service Market Op...,Artificial Intelligence Service Market Opportu...,Artificial Intelligence Service Market Opportu...,Artificial Intelligence Service Market Opportu...


In [71]:
## finding relevant topics

In [10]:
ai_keywords = ['Data Science', 'data science', 'DATA SCIENCE', 'AI', 'ai', 'artificial intelligence', 'Artificial Intelligence',
            'ARTIFICIAL INTELLIGENCE', 'ML', 'NLP', 'Artificial General Intelligence', 'Chatbots', 'AI Marketplaces', 
            'Intelligent Applications', 'Augmented Intelligence', 'Decision Intelligence', 'AI Cloud Services', 'cloud services', 
            'GPU Accelerators', 'Computer Vision', 'Deep Neural Network', 'Deep Learning', 'Cognitive Computing', 
            'Knowledge Graphs', 'Responsible AI', 'Machine Customers', 'Decision Intelligence', 
            'Human-Centered AI', 'AI Governance', 'Natural Language Processing', 'Machine Learning', 'Smart Robots', 
            'Operational AI Systems', 'Data-Centric AI', 'AI TRiSM', 'Generative AI', 'Responsible AI', 'OpenAI', 
            'openai', 'gpt', 'GPT', 'technology', 'analytics']

keywords_lower = [keyword.lower() for keyword in ai_keywords]

def clean_text(text):
    text_lower = text.lower()
    for keyword in keywords_lower:
        text_lower = text_lower.replace(keyword, '')
    return text_lower

df2['clean_text_no_ai'] = df2['clean_text_v5'].apply(clean_text)

In [11]:
df2.head()

,date,clean_title_v3,clean_text_v3,clean_text_v4,clean_text_v5,clean_text_no_ai
0,2020-02-26,LegalTech Artificial Intelligence Market Tech...,LegalTech Artificial Intelligence Market Tech...,LegalTech Artificial Intelligence Market Techn...,LegalTech Artificial Intelligence Market Techn...,legaltech market advancement future scope ca...
2,2020-03-13,Covid un messaggio dai ricercatori italiani ai...,Covid un messaggio dai ricercatori italiani ai...,Covid un messaggio dai ricercatori italiani ai...,Covid un messaggio dai ricercatori italiani ai...,covid un messaggio d ricercatori italiani col...
7,2021-02-25,MulticoreWare Inc Becomes CEVAs Trusted Partne...,MulticoreWare Inc Becomes CEVAs Trusted Partne...,MulticoreWare Inc Becomes CEVAs Trusted Partne...,MulticoreWare Inc Becomes CEVAs Trusted Partne...,multicoreware inc becomes cevas trusted partne...
8,2022-10-06,Healthcare Artificial Intelligence Market Anal...,Healthcare Artificial Intelligence Market Anal...,Healthcare Artificial Intelligence Market Anal...,Healthcare Artificial Intelligence Market Anal...,healthcare market analysis forecast next year...
9,2020-04-29,Artificial Intelligence As A Service Market Op...,Artificial Intelligence Service Market Opportu...,Artificial Intelligence Service Market Opportu...,Artificial Intelligence Service Market Opportu...,service market opportunities market analysis ...


In [12]:
df2 = df2[['date', 'clean_title_v3', 'clean_text_v5', 'clean_text_no_ai']]

In [13]:
df2.head()

,date,clean_title_v3,clean_text_v5,clean_text_no_ai
0,2020-02-26,LegalTech Artificial Intelligence Market Tech...,LegalTech Artificial Intelligence Market Techn...,legaltech market advancement future scope ca...
2,2020-03-13,Covid un messaggio dai ricercatori italiani ai...,Covid un messaggio dai ricercatori italiani ai...,covid un messaggio d ricercatori italiani col...
7,2021-02-25,MulticoreWare Inc Becomes CEVAs Trusted Partne...,MulticoreWare Inc Becomes CEVAs Trusted Partne...,multicoreware inc becomes cevas trusted partne...
8,2022-10-06,Healthcare Artificial Intelligence Market Anal...,Healthcare Artificial Intelligence Market Anal...,healthcare market analysis forecast next year...
9,2020-04-29,Artificial Intelligence As A Service Market Op...,Artificial Intelligence Service Market Opportu...,service market opportunities market analysis ...


In [14]:
df2.to_parquet(r'C:\Users\Rahul\Downloads\NLP Final Project\Preprocessing_3_no_ai.parquet', engine='pyarrow')